In [1]:
import pandas as pd
import datetime
import re

In [2]:
# load dữ liệu vào dataframe
data = pd.read_csv("D:\\DA_Camp\\adjusted_data.csv")
visitor = pd.read_csv("D:\\DA_Camp\\visitor_information.csv")

# **1. Kiểm tra liệu tất cả Id trong data đều có trong visitor_information?**

In [3]:
# Tạo list visitor_only gồm những id chỉ có trong visitor nhưng không có trong data
visitor_only = []
a = data['visitor_id'].unique()
for i in visitor['visitor_id'].unique():
    if i not in a:
        visitor_only.append(i)

In [5]:
# Print ra số lượng id của visitor_only (những người chỉ ghé nhưng chưa mua), buyer (những người đã mua), và cả 2 
print("visitor_only: " + str(len(visitor_only)))
print("vistitor_only + buyer: " + str(len(visitor['visitor_id'].unique())))
print("buyer: " + str(len(a)))

visitor_only: 86964
vistitor_only + buyer: 126403
buyer: 39440


In [6]:
# So sánh thấy lệch 1 id và đó là do id "Khách vãng lai" ở trong bảng data
39440 + 86964

126404

In [40]:
# Em tò mò những người visitor_only này biết công ty qua những nguồn nào bởi vì nếu từ cửa hàng thì hơi kỳ vì đến cửa hàng, lưu thông tin lại mà không mua
# Lạ là có visitor_only từ cửa hàng thật =))) 
b= []
for i in visitor.index:
    if visitor['visitor_source'][i] in b:
        continue
    if visitor['visitor_id'][i] in visitor_only:
        b.append(visitor['visitor_source'][i])
b

['Website công ty', 'Sàn thương mại điện tử', 'Cừa hàng']

# **2. Liệu có phải mọi Id trong file data đều đã mua hàng ít nhất 1 lần?**

In [ ]:
'''
1. Sắp xếp bảng data theo Id. Mỗi Id sẽ tạo thành 1 khối 100 ngày
2. Loop qua từng khối
3. Trong mỗi khối loop qua từng ngày
4. Khối nào có noOrders = 0 trong cả 100 ngày thì print ra Id. Nếu không có gì được print ra thì chứng tỏ mọi Id trong file data đã mua hàng ít nhất 1 lần
'''

In [7]:
data_sort =  data.sort_values(by=['visitor_id', 'create_date'], ascending= [1,1], ignore_index= True)
data_sort

,visitor_id,noOrders,create_date
0,++DZ24GqVhNE0Bl7f3Zp06lWjj4=,0,2019-11-01
1,++DZ24GqVhNE0Bl7f3Zp06lWjj4=,0,2019-11-02
2,++DZ24GqVhNE0Bl7f3Zp06lWjj4=,0,2019-11-03
3,++DZ24GqVhNE0Bl7f3Zp06lWjj4=,0,2019-11-04
4,++DZ24GqVhNE0Bl7f3Zp06lWjj4=,0,2019-11-05
...,...,...,...
3943995,zzxiPB4vtRlO3eKR9TsP2+iQapc=,0,2020-02-04
3943996,zzxiPB4vtRlO3eKR9TsP2+iQapc=,0,2020-02-05
3943997,zzxiPB4vtRlO3eKR9TsP2+iQapc=,0,2020-02-06
3943998,zzxiPB4vtRlO3eKR9TsP2+iQapc=,0,2020-02-07


In [ ]:
for i in range(0, 3944000, 100):
    c = 0
    for j in range (i,i+100):
        if data_sort['noOrders'][j] == 0:
            c += 1
    if c == 100: 
        print(data_sort.loc[i])

# **3. Nếu 43770 tương đương 1/11/2019 thì có ai mua hàng lần đầu trước khi biết đến công ty không?**

## *Lúc này em chưa biết là 1/11/2019 khi thay đổi format từ date sang number thì nó sẽ là 43770. Vì vậy em dùng luận điểm này để chứng minh 43770 tương đương với 1/11/2019*


In [9]:
'''
1. Từ bảng data lấy ra lần đầu mua hàng của mỗi id 
    - Từ bảng data, bỏ đi những ngày không mua hàng của tất cả id
    - Sắp xếp phần còn lại theo id và create_date theo thứ tự tăng dần
    - Vì đã sắp xếp như vậy nên để lấy lần đầu mua hàng của 1 id thì chỉ cần bỏ duplicate theo cột id và giữ lại dòng đầu tiên của mỗi nhóm id trùng
2. Sau đó merge với bảng visitor_information để có thêm ngày đầu tiên visit của các id
3. So sánh ngày đầu mua và ngày đầu visit
'''

'\n1. Từ bảng data lấy ra lần đầu mua hàng của mỗi id \n    - Bỏ đi những ngày không mua hàng của tất cả id\n    - Sort theo id và create_date theo thứ tự tăng dần\n    - Vì đã sắp xếp như vậy nên để lấy lần đầu mua hàng của 1 id thì chỉ cần bỏ duplicate theo cột id và giữ lại dòng đầu tiên của mỗi nhóm id trùng đấy\n2. Sau đó merge với bảng visitor_information để có thêm ngày đầu tiên visit của các id\n3. So sánh ngày đầu mua và ngày đầu visit\n'

In [44]:
# Từ bảng data, bỏ đi những ngày không mua hàng của tất cả id
data_oldcust = data.drop(labels= data.loc[data['noOrders'] == 0].index)
data_oldcust

,visitor_id,noOrders,create_date
2840311,++DZ24GqVhNE0Bl7f3Zp06lWjj4=,2,2020-01-12
947193,++HN809MR7t8V9QmdOEA36NubfI=,2,2019-11-25
316154,++J4GEqZUrL8Rf7XrURsNNt6KPw=,1,2019-11-09
434476,++P59XK9q+K1acy8MeRVzfrfOz0=,1,2019-11-12
473916,++P59XK9q+K1acy8MeRVzfrfOz0=,2,2019-11-13
...,...,...,...
1735337,zzGUju7RRo8FrQOIOmNM9vgwuns=,1,2019-12-14
1064873,zzURsuIJlSzPj3lOz19YWU1VMrE=,1,2019-11-27
552156,zzWdLg/Dyw9eyko6k+ZSDUuTio0=,1,2019-11-14
631036,zzWdLg/Dyw9eyko6k+ZSDUuTio0=,1,2019-11-16


In [ ]:
# Sắp xết phần còn lại theo id và create_date theo thứ tự tăng dần
data_oldcust = data_oldcust.sort_values(by=['visitor_id', 'create_date'], ascending= [1,1])
data_oldcust


In [ ]:
# Ta đang làm việc với các Id có thông tin nên xóa Id "Khách vãng lai" 
data_oldcust = data_oldcust.drop(labels= data.loc[data['visitor_id'] == "Khách vãng lai"].index)
data_oldcust

In [ ]:
# Lấy ra thông tin lần đầu mua hàng bằng cách bỏ duplicate và giữ lại dòng đầu trong nhóm bị duplicate
first_time = data_oldcust.drop_duplicates(subset= "visitor_id", keep="first")
first_time

In [46]:
# Sắp xếp lại index và sửa tiêu đề cột cho đúng
first_time = first_time.sort_values(by=['visitor_id', 'create_date'], ascending= [1,1], ignore_index= True)
first_time = first_time.rename({'create_date' : 'first_buy'},axis=1)
first_time

,visitor_id,noOrders,first_buy
0,++DZ24GqVhNE0Bl7f3Zp06lWjj4=,2,2020-01-12
1,++HN809MR7t8V9QmdOEA36NubfI=,2,2019-11-25
2,++J4GEqZUrL8Rf7XrURsNNt6KPw=,1,2019-11-09
3,++P59XK9q+K1acy8MeRVzfrfOz0=,1,2019-11-12
4,++RP+syfDvZiTf/Pmov3PdEeFWM=,3,2019-12-04
...,...,...,...
39434,zz4YKQwMrdb6za9UWGnbxWHcLZY=,1,2019-11-08
39435,zzGUju7RRo8FrQOIOmNM9vgwuns=,1,2019-12-14
39436,zzURsuIJlSzPj3lOz19YWU1VMrE=,1,2019-11-27
39437,zzWdLg/Dyw9eyko6k+ZSDUuTio0=,1,2019-11-14


In [47]:
# Đổi tiêu đề cột ở dataframe 'visitor' cho đúng
visitor = visitor.rename({'create_date' : 'first_visit'}, axis= 1)
visitor

,first_visit,visitor_id,gender,age,visitor_source
0,2020-02-08,eLbvQsgdm1yEo44ggZA6i9mXYIg=,Nam,37.0,Cừa hàng
1,2020-02-08,5I6he68XtIQHpsRGKp0EsfyM3x4=,LGBT,0.0,Website công ty
2,2020-02-08,CP4VK05fHSxd8oMQ2TPPKp49ZtQ=,Nam,0.0,Website công ty
3,2020-02-08,k8lYxsZaAhgnQOgTTgFlVvw6GGQ=,Nữ,0.0,Website công ty
4,2020-02-08,x65MkmEJiaGCq3OGxuI2uvfq4os=,Không rõ,39.0,Sàn thương mại điện tử
...,...,...,...,...,...
126398,2020-02-04,UaPkbFX9VqV4szba0Ya/E5SFKS4=,Nam,35.0,Cừa hàng
126399,2020-02-05,y3cDfPPmSLJRtPGL0/pExUOAqeQ=,Không rõ,35.0,Sàn thương mại điện tử
126400,2020-02-06,snYIf8MqQkimpm2DpiTnXHx8IoM=,Nam,35.0,Cừa hàng
126401,2020-02-06,42YVqRkkg0q2PNI4UkLLdKY7SYs=,Nam,35.0,Cừa hàng


In [48]:
# merge hai bảng và giữ lại các id ở bảng first_time
first_time = pd.merge(first_time, visitor, how= 'left', on = 'visitor_id')
first_time

,visitor_id,noOrders,first_buy,first_visit,gender,age,visitor_source
0,++DZ24GqVhNE0Bl7f3Zp06lWjj4=,2,2020-01-12,2020-01-10,LGBT,21.0,Website công ty
1,++HN809MR7t8V9QmdOEA36NubfI=,2,2019-11-25,2019-11-25,Nữ,17.0,Website công ty
2,++J4GEqZUrL8Rf7XrURsNNt6KPw=,1,2019-11-09,2019-11-09,Nam,1.0,Website công ty
3,++P59XK9q+K1acy8MeRVzfrfOz0=,1,2019-11-12,2019-11-12,Nữ,19.0,Website công ty
4,++RP+syfDvZiTf/Pmov3PdEeFWM=,3,2019-12-04,2019-12-04,Không rõ,NaN,Website công ty
...,...,...,...,...,...,...,...
39434,zz4YKQwMrdb6za9UWGnbxWHcLZY=,1,2019-11-08,2019-11-01,Nữ,NaN,Website công ty
39435,zzGUju7RRo8FrQOIOmNM9vgwuns=,1,2019-12-14,2019-12-14,Không rõ,NaN,Website công ty
39436,zzURsuIJlSzPj3lOz19YWU1VMrE=,1,2019-11-27,2019-11-27,Không rõ,NaN,Website công ty
39437,zzWdLg/Dyw9eyko6k+ZSDUuTio0=,1,2019-11-14,2019-11-14,Không rõ,NaN,Website công ty


In [49]:
# Nhận thấy bảng first_time lúc này có thể coi như bảng thông tin của khách hàng, sau có thể dùng nên chỉnh lại các chỗ 'NaN' ở cột 'age' luôn
first_time['age'] = first_time['age'].fillna('Không rõ')
first_time

,visitor_id,noOrders,first_buy,first_visit,gender,age,visitor_source
0,++DZ24GqVhNE0Bl7f3Zp06lWjj4=,2,2020-01-12,2020-01-10,LGBT,21.0,Website công ty
1,++HN809MR7t8V9QmdOEA36NubfI=,2,2019-11-25,2019-11-25,Nữ,17.0,Website công ty
2,++J4GEqZUrL8Rf7XrURsNNt6KPw=,1,2019-11-09,2019-11-09,Nam,1.0,Website công ty
3,++P59XK9q+K1acy8MeRVzfrfOz0=,1,2019-11-12,2019-11-12,Nữ,19.0,Website công ty
4,++RP+syfDvZiTf/Pmov3PdEeFWM=,3,2019-12-04,2019-12-04,Không rõ,Không rõ,Website công ty
...,...,...,...,...,...,...,...
39434,zz4YKQwMrdb6za9UWGnbxWHcLZY=,1,2019-11-08,2019-11-01,Nữ,Không rõ,Website công ty
39435,zzGUju7RRo8FrQOIOmNM9vgwuns=,1,2019-12-14,2019-12-14,Không rõ,Không rõ,Website công ty
39436,zzURsuIJlSzPj3lOz19YWU1VMrE=,1,2019-11-27,2019-11-27,Không rõ,Không rõ,Website công ty
39437,zzWdLg/Dyw9eyko6k+ZSDUuTio0=,1,2019-11-14,2019-11-14,Không rõ,Không rõ,Website công ty


In [50]:
# Em tò mò xem có bao nhiều trường hợp visit trước và mua sau
first_time.loc[first_time['first_buy'] != first_time['first_visit']]

,visitor_id,noOrders,first_buy,first_visit,gender,age,visitor_source
0,++DZ24GqVhNE0Bl7f3Zp06lWjj4=,2,2020-01-12,2020-01-10,LGBT,21.0,Website công ty
5,++Rc3Z/cGqApeiipO78fSuUXrdo=,1,2019-12-24,2019-12-23,Nữ,28.0,Cừa hàng
8,++sf+nIMstu6uI9e53II6GyjGQU=,1,2019-12-02,2019-11-29,LGBT,18.0,Website công ty
12,+/LFbKVyPDid530ZMAXN/TX3hxw=,2,2019-11-10,2019-11-01,Nam,32.0,Website công ty
14,+/fUxAU5gfFaBfWV5YZTfxgMMHY=,1,2020-01-27,2020-01-26,Nam,0.0,Website công ty
...,...,...,...,...,...,...,...
39418,zxZ9G59CMhNu6dxAnWSg3A6cJGA=,2,2019-12-20,2019-12-09,Nam,32.0,Website công ty
39424,zy1hOBgaWshDk1K3SfLiH+BMSSI=,1,2020-01-16,2020-01-12,Không rõ,20.0,Website công ty
39426,zyDp+AbZP4m9TnqwKbMuPYy11iU=,1,2019-12-25,2019-12-24,Không rõ,Không rõ,Website công ty
39427,zyI5xVs7Av/37rWmlrJ3kIHzjkQ=,1,2019-11-20,2019-11-03,Nam,25.0,Website công ty


In [52]:
# Kiểm tra xem có trường hợp nào first_time < create_date không
tdelta = datetime.timedelta(days = 0)
for i in first_time.index:
    a = datetime.datetime.strptime(first_time['first_buy'][i],"%Y-%m-%d").date()
    b = datetime.datetime.strptime(first_time['first_visit'][i],"%Y-%m-%d").date()
    if a - b < tdelta:
        print("Not Good")

In [53]:
# Lưu lại bảng first_time để dùng sau
first_time.to_csv("D:\\DA_Camp\\customer_information.csv",index = False)

# **Khi làm câu c, nhận thấy cần phải dùng bảng data_oldcust. Vì vậy nên cũng lưu lại

In [54]:
data_oldcust.to_csv("D:\\DA_Camp\\old_cust.csv",index=False)